# Week 6 Problem Set

## Homeworks

**HW1.** Extend the class `Fraction` to implement the other operators: `- * < <= > >=`.

In [1]:
def gcd(a, b):
    if b == 0:
        return a
    else:
        return gcd(b, a % b)


class Fraction:
    # copy the rest of the methods here
    
    def __init__(self, num, den):
        self._num = int(num)
        self._den = int(den) if den != 0 else 1
    
    @property
    def num(self):
        return self._num
    
    @num.setter
    def num(self, val):
        self._num = int(val)

    @property
    def den(self):
        return self._den
    
    @den.setter
    def den(self, val):
        self._den = val if val != 0 else 1
    
    def __str__(self):
        return f"{self.num}/{self.den}"
    
    def simplify(self):
        divisor = gcd(self.num, self.den)
        return Fraction(self.num / divisor, self.den / divisor)
    
    def __add__(self, other):
        return Fraction(self.num * other.den + other.num * self.den, self.den * other.den).simplify()
        
    
    def __eq__(self, other):
        self_simplified = self.simplify()
        other_simplified = other.simplify()
        return self_simplified.num == other_simplified.num and self_simplified.den == other_simplified.den
    

    
    def __sub__(self, other):
        return Fraction(self.num * other.den - other.num * self.den, self.den * other.den).simplify()
    
    def __mul__(self, other):
        return Fraction(self.num * other.num, self.den * other.den).simplify()
    
    def __truediv__(self, other):
        return Fraction(self.num * other.den, self.den * other.num).simplify()
    
    def __lt__(self, other):
        return (self.num / self.den) < (other.num / other.den)
    
    def __le__(self, other):
        return (self.num / self.den) <= (other.num / other.den)
    
    def __gt__(self, other):
        return not (self <= other)
    
    def __ge__(self, other):
        return not (self < other)
    


In [2]:
f1 = Fraction(3, 4)
f2 = Fraction(1, 2)
f3 = f1 - f2
assert f3 == Fraction(1, 4)
f4 = f1 * f2
assert f4 == Fraction(3, 8)
assert f2 < f1
assert f2 <= f2
assert f1 > f3
assert f3 >= f3

In [3]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [4]:
class MixedFraction(Fraction):
    def __init__(self, top, bot, whole=0):
        num = whole * bot + top
        super().__init__(num, bot)

    def get_three_numbers(self):
        whole = self.num // self.den
        top = self.num % self.den
        bot = self.den
        return (top, bot, whole)

    def __str__(self):
        top, bot, whole = self.get_three_numbers()
        if whole == 0:
            return f"{top}/{bot}"
        else:
            return f"{whole} {top}/{bot}"


In [5]:
mf1 = MixedFraction(5, 3)
assert mf1.num == 5 and mf1.den == 3
assert mf1.get_three_numbers() == (2, 3, 1)
mf2 = MixedFraction(2, 3, 1)
assert mf2.num == 5 and mf2.den == 3

result = mf1 + mf2
assert result.num == 10 and result.den == 3

result = mf1 * mf2
assert result.num == 25 and result.den == 9

mf3 = MixedFraction(1, 2, 1)
result = mf1 - mf3
assert result.num == 1 and result.den == 6

assert str(mf1) == "1 2/3"

In [6]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


**HW2.** Write a class called `EvaluateFraction` that evaluates postfix notation implemented using Dequeue data structures only. Postfix notation is a way of writing expressions without using parenthesis. For example, the expression `(1+2)*3` would be written as `1 2 + 3 *`. The class `EvaluateFraction` has the following method:
- `input(inp)`: which pushes the input input one at a time. For example, to create a postfix notation `1 2 + 3 *`, we can call this method repetitively, e.g. `e.input('1'); e.input('2'); e.input('+'); e.input('3'); e.input('*')`. Notice that the input is of String data type. 
- `evaluate()`: which returns the output of the expression.
- `get_fraction(inp)`: which takes in an input string and returns a `Fraction` object. 

Postfix notation is evaluated using a Stack. Since `Dequeue` can be used for both Stack and Queue, we will implement using `Dequeue`. The input streams from `input()` are stored in a Queue, which we will again implement using Dequeue. If the output of the Queue is a number, the item is pushed into the stack. If it is an operator, we will apply the operator to the two top most item n the stacks and push the result back into the stack. 

In [7]:
#Copy the Cohort implementation
# my brother in christ, this has been copy-pasted 4 times now
# could these problem sets not have been presented as like
# something other than jupyter notebooks?
# like just a set of files would've been nice
# the idea of slowly building up a library `ddw.py` would've been cool
# like a library of data structures like `Stack`, `Queue`, `mergesort`, etc...
# which is what this mod looks like it's doing, right?
# anyway, this is copied over from week 4 cohort ::sigh::

class Stack:
    def __init__(self):
        self._items = []
        
    def push(self, item):
        self._items.append(item)

    def pop(self):
        if self.is_empty:
            return None
        return self._items.pop()

    def peek(self):
        if self.is_empty:
            return None
        return self._items[-1]

    @property
    def is_empty(self):
        return self.size == 0

    @property
    def size(self):
        return len(self._items)

In [8]:
class Queue:
    def __init__(self):
        self.left_stack = Stack()
        self.right_stack = Stack()
    
    def enqueue(self, new_item):
        self.right_stack.push(new_item)

    def dequeue(self):
        if self.left_stack.is_empty:
            self.left_stack._items = self.right_stack._items[::-1]
            self.right_stack._items.clear()
        return self.left_stack.pop()

    def peek(self):
        if self.is_empty:
            return None
        elif self.left_stack.is_empty:
            return self.right_stack._items[0]
        else:
            return self.left_stack.peek()
    
    @property
    def is_empty(self):
        return self.left_stack.is_empty and self.right_stack.is_empty
    
    @property
    def size(self):
        return self.left_stack.size + self.right_stack.size
        
    def __repr__(self):
        return ", ".join([str(i) for i in self.left_stack._items[::-1] + self.right_stack._items])



In [9]:
# dude, this could've been like
# lib/week6_cohort.py
# then the homework problem could've been just week6_homework.py
# then we wouldn't have to keep copy-pasting stuff around
# what happened to modularity man ::sadge::

class Deque(Queue):
  
    def add_front(self, item):
        self.left_stack.push(item)
      
    def remove_front(self):
        return self.dequeue()
            
    def add_rear(self, item):
        self.enqueue(item)
    
    def left_to_right(self):
        self.right_stack._items = self.left_stack._items[::-1]
        self.left_stack._items.clear()
    
    def remove_rear(self):
        if self.right_stack.is_empty:
            self.left_to_right()
        return self.right_stack.pop()
    
    def peek_front(self):
        if self.left_stack.is_empty:
            return self.right_stack._items[0] if not self.right_stack.is_empty else None
        else:
            return self.left_stack.peek()
    
    def peek_rear(self):
        if not self.right_stack.is_empty:
            return self.right_stack.peek()
        else:
            return self.left_stack._items[0] if not self.left_stack.is_empty else None

In [10]:
class EvaluateFraction:

    operands = "0123456789"
    operators = {
        "+": lambda a, b: a + b,
        "-": lambda a, b: a - b,
        "*": lambda a, b: a * b,
        "/": lambda a, b: a / b
    }
    
    def __init__(self):
        self.expression = Deque()
        self.stack = Deque()
    
    def input(self, item):
        self.expression.add_rear(item)

    
    def evaluate(self):
        while not self.expression.is_empty:
            char = self.expression.remove_front()
            if char in self.operators.keys():
                b = self.get_fraction(self.stack.remove_front())
                a = self.get_fraction(self.stack.remove_front())
                self.stack.add_front(self.operators[char](a, b))
            else:
                self.stack.add_front(char)
        return self.stack.remove_front()

    
    def get_fraction(self, inp):
        if isinstance(inp, Fraction):
            return inp
        return Fraction(*(int(i.strip()) for i in inp.split("/")))

    
    # def process_operator(self, op1, op2, op):
    #     ###
    #     ### YOUR CODE HERE
    #     ###


In [11]:
pe = EvaluateFraction()
pe.input("1/2")
pe.input("2/3")
pe.input("+")
assert pe.evaluate()==Fraction(7, 6)

pe.input("1/2")
pe.input("2/3")
pe.input("+")
pe.input("1/6")
pe.input("-")
assert pe.evaluate()==Fraction(1, 1)

pe.input("1/2")
pe.input("2/3")
pe.input("+")
pe.input("1/6")
pe.input("-")
pe.input("3/4")
pe.input("*")
assert pe.evaluate()==Fraction(3, 4)

In [12]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


**HW3.** Modify HW2 so that it can work with MixedFraction. Write a class called `EvaluateMixedFraction` as a subclass of `EvaluateFraction`. You need to override the following methods:
- `get_fraction(inp)`: This function should be able to handle string input for MixedFraction such as `1 1/2` or `3/2`. It should return a `MixedFraction` object.
- `evaluate()`: This function should return `MixedFraction` object rather than `Fraction` object. 

In [13]:
class EvaluateMixedFraction(EvaluateFraction):
    def get_fraction(self, inp):
        if isinstance(inp, Fraction):
            return inp
        mixed, denominator = inp.split("/")
        if " " in mixed:
            whole, numerator = mixed.split(" ")
            numerator, denominator, whole = int(numerator), int(denominator), int(whole)
            return MixedFraction(top=numerator, bot=denominator, whole=whole)
        else:
            numerator, denominator = int(mixed), int(denominator)
            return MixedFraction(top=numerator, bot=denominator)
    
    def evaluate(self):
        # muahahahaha no need to override
        return super().evaluate()

In [14]:
pe = EvaluateMixedFraction()
pe.input("3/2")
pe.input("1 2/3")
pe.input("+")
assert pe.evaluate() == MixedFraction(1, 6, 3)

pe.input("1/2")
pe.input("2/3")
pe.input("+")
pe.input("1 1/8")
pe.input("-")
assert pe.evaluate() == MixedFraction(1, 24)

pe.input("1 1/2")
pe.input("2 2/3")
pe.input("+")
pe.input("1 1/6")
pe.input("-")
pe.input("5/4")
pe.input("*")
assert pe.evaluate() == MixedFraction( 3, 4, 3)

In [15]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


**HW4.** *Linked List:* We are going to implement Linked List Abstract Data Type. To do so, we will implement two classes: `Node` and `MyLinkedList`. In this part, we will implement the class Node.

The class `Node` has the following attribute and computed property:
- `element`: which stores the value of the item in that node.
- `next`: which stores the reference to the next `Node` in the list. The setter method should check if the value assigned is of type `Node`.





In [23]:
class Node:
    def __init__(self, e):
        self.element = e
        self.__next = None
               
    @property
    def next(self):
        return self.__next
    
    @next.setter
    def next(self, value):
        if isinstance(value, Node) or value is None:
            self.__next = value
        else:
            raise TypeError("next must be a Node")
    
    def __eq__(self, other):
        return self.element == other
    
    def __repr__(self):
        return f"Node({self.element})"

    def __str__(self):
        return f"Node({self.element})"


In [17]:
n1 = Node(1)
n2 = Node(2)
n3 = Node(3)
assert n1.element == 1 and n2.element == 2 and n3.element == 3
n1.next = n2
n2.next = n3
assert n1.next == n2 and n2.next == n3

In [18]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


**HW5.** This is a continuation to implement a Linked List. The class `MyLinkedList` has two different properties:
- `head`: which points to the `Node` of the first element.
- `tail`: which points to the `Node` of the last element.

It should also have the following methods:
- `__init__(items)`: which create the link list object based using the arguments.
- `_get(index)`: which returns the item at the given `index`.
- `_add_first(item)`: which adds the `item` as the first element.
- `_add_last(item)`: which adds the `item` as the last element.
- `_add_at(index, item)`: which adds the `item` at the position `index`.
- `_remove_first(item)`: which removes the `item` as the first element.
- `_remove_last(item)`: which removes the `item` as the last element.
- `_remove_at(index, item)`: which removes the `item` at the position `index`.
- `_index_of(item)`: which returns the index of the given item and is called by `remove(item)` in the parent class.


In [19]:

import collections.abc as c
from abc import abstractmethod

class MyAbstractList(c.Iterator):

    
    def __init__(self, list_items):
        # iterate over every element and call self.add(item)
        self.size = 0
        self._idx = 0
        for item in list_items:
            self.append(item)
    
    
    @property
    def is_empty(self):
        return self.size == 0
    
    def append(self, item):
        # call add_at() method here
        self._add_at(self.size, item)
        
    def remove(self, item):
        # you should use remove_at() method here
        idx = self._index_of(item)
        if  idx >= 0:
            self._remove_at(idx)
            return True
        else:
            return False
        
    def __getitem__(self, index):
        return self._get(index)
    
    def __setitem__(self, index, value):
        # call set_at(index, value) method here
        self._set_at(index, value)
        
    def __delitem__(self, index):
        # call remove_at() method here
        self._remove_at(index)
    
    def __len__(self):
        return self.size
        
    def __iter__(self):
        self._idx = 0
        return self
        
    def __next__(self):
        if self._idx < self.size:
            n_item = self._get(self._idx)
            self._idx += 1
            return n_item
        else:
            raise StopIteration
    
    def __repr__(self):
        return f"{type(self).__name__}( {', '.join((str(i) for i in self))} )"
    
    # the following methods should be implemented in the child class
    
    @abstractmethod
    def _get(self, index):
        pass

    @abstractmethod
    def _set_at(self, index, item):
        pass

    @abstractmethod
    def _add_at(self, index, item):
        pass

    @abstractmethod
    def _remove_at(self, index):
        pass

    @abstractmethod
    def _index_of(self, item):
        pass

In [26]:
class MyLinkedList(MyAbstractList):
    def __init__(self, items):
        self.head: Node | None = None
        self.tail: Node | None = None
        super().__init__(items)
        
    def _get(self, index) -> Node:
        res = self.head
        for _ in range(index):
            if res is None:
                return None
            res = res.next
        return res
    
    def _add_first(self, element):
        # do the following:
        # 1. create a new Node object using element
        # 2. set the current head reference as the next reference of the new node
        # 3. increase size by 1
        # 4. if this is the last element (no tail) -> set the current node as the tail
        new_node = Node(element)
        new_node.next = self.head
        self.head = new_node
        if self.tail is None:
            self.tail = new_node
        self.size += 1
            
    def _add_last(self, element):
        # do the following:
        # 1. create a new Node object using element
        # 2. if there is no element as tail -> set the new node as both
        #    the tail and the head
        # 3. otherwise, -> 
        #    - set the new node as the next reference of the tail
        #    - set the next reference of the current node as the tail's next reference
        # 4. increase size by 1
        new_node = Node(element)
        if self.tail is None:
            self.head = new_node
            self.tail = new_node
        else:
            self.tail.next = new_node
            self.tail = new_node
        self.size += 1
        
    def _add_at(self, index, element):
        if index == 0:
            # if insert at first position, call add_first() method
            self._add_first(element)
        elif index >= self.size:
            # if insert at last position, call add_last() method
            self._add_last(element)
            pass
        else:
            # if insert in between, do the following:
            # 1. start from the head, traverse the linked list to get
            #    the reference at position index-1 using its next reference
            # 2. create a new Node
            # 3. set the next of the current node as the next of the new Node
            # 4. set the new node as the next of the current node
            # 5. increase the size by 1
            new_node = Node(element)
            previous_node = self._get(index - 1)
            next_node = previous_node.next
            previous_node.next = new_node
            new_node.next = next_node
            self.size += 1
            
    def _set_at(self, index, element):
        if index in range(0, self.size):
            current = self._get(index)
            current.element = element
            
    def _remove_first(self):
        if self.size == 0:
            # if list is empty, return None
            return None
        else:
            # otherwise, do the following:
            # 1. store the head at a temporary variable
            # 2. set the next reference of the current head to be the head
            # 3. reduce size by 1
            # 4. if the new head is now None, it means empty list
            #    -> set the tail to be None also
            # 5. return element of the removed node
            res = self.head.element
            self.head = self.head.next
            self.size -= 1
            if self.head is None:
                self.tail = None
            return res
        
    def _remove_last(self):
        if self.size == 0:
            # if the list is empty, return None
            return None
        elif self.size == 1:
            # if there is only one element, just remove that one node 
            # using some other method
            self._remove_first()
        else:
            # otherwise, do the following:
            # 1. traverse to the second last node
            # 2. store the tail of the list to a temporary variable
            # 3. set the current node as the tail
            # 4. set the next ref of the tail to be None
            # 5. reduce the size by 1
            # 6. return the element of the removed node in the temp var
            new_last_node = self._get(self.size - 2)
            res = self.tail.element
            new_last_node.next = None
            self.tail = new_last_node
            self.size -= 1
            return res
        
    def _remove_at(self, index):
        if index < 0 or index >= self.size:
            return None
        elif index == 0:
            return self._remove_first()
        elif index == self.size - 1:
            return self._remove_last()
        else:
            # do the following:
            # 1. traverse to the node at index - 1
            # 2. get the node at index using next reference
            # 3. set the next node of the node at index - 1
            # 4. decrease the size by 1
            # 5. return the element that is removed
            previous_node = self._get(index - 1)
            node_to_remove = self._get(index)
            next_node = self._get(index + 1)

            previous_node.next = next_node
            self.size -= 1
            return node_to_remove.element
            
    
    def _index_of(self, item):
        # do the following:
        # 1. Initialize index to 0 and current node to the head node
        # 2. Iterate to the end of the linked list
        # 3. if the item is the same as the current node's element, return the current index
        # 4. otherwise, increase the index and move current node to the next element
        # 5. If we loop to the end and have not exit, return -1
        current_node = self.head
        i = 0
        while current_node.next is not None:
            if current_node.element == item:
                return i
            else:
                current_node = current_node.next
                i += 1
        return i
    
    


In [27]:
asean = MyLinkedList(['Singapore', 'Malaysia'])
assert asean.head.element == 'Singapore'
assert asean.tail.element == 'Malaysia'

asean.append('Indonesia')
assert asean.tail.element == 'Indonesia'
asean._add_at(0, 'Brunei')
assert asean.head.element == 'Brunei'
assert asean.size == 4
assert len(asean) == 4
assert asean.remove('Singapore')
assert len(asean) == 3
assert asean[1] == 'Malaysia'
asean._add_at(1, 'Singapore')

asean[0] = 'Cambodia'
assert asean[0] == 'Cambodia' and asean[1] == 'Singapore'
asean[2] = 'Myanmar'
assert(len(asean)) == 4 
assert [x for x in asean] == ['Cambodia', 'Singapore', 'Myanmar', 'Indonesia']


del asean[0]
assert [x for x in asean] == ['Singapore', 'Myanmar', 'Indonesia']

asean._add_at(2, 'Brunei')
assert [x for x in asean] == ['Singapore', 'Myanmar', 'Brunei', 'Indonesia']
del asean[3]
assert [x for x in asean] == ['Singapore', 'Myanmar', 'Brunei']
del asean[1]
assert [x for x in asean] == ['Singapore', 'Brunei']
del asean[1]
assert [x for x in asean] == ['Singapore']
del asean[0]
assert [x for x in asean] == []

In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###
